#### 1. Start a Spark Session.

In [49]:
from pyspark.sql.functions import format_number, dayofyear, mean, year, min, max, corr, month
from pyspark.sql import SparkSession

In [50]:
spark = SparkSession.builder.appName('Project').getOrCreate()

#### 2. Load the Walmart Stock CSV File, have Spark infer the data types.

In [51]:
df = spark.read.csv('walmart_stock.csv', inferSchema = True, header = True)
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

#### 3. What are the column names?

In [52]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### 4. What does the Schema look like?

In [53]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### 5. Print out the first five columns.

In [54]:
for row in df.head(5):
    print(row)
    print('')

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)

Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)

Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)

Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)

Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)



#### 6. Use describe() to learn about the DataFrame.

In [55]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

#### 7. There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes.

In [56]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [57]:
df1 = df.describe()
df1.select(df1['summary'], 
           format_number(df1['Open'].cast('float'),2).alias('Open'),
           format_number(df1['High'].cast('float'),2).alias('High'),
           format_number(df1['Low'].cast('float'),2).alias('Low'),
           format_number(df1['Close'].cast('float'),2).alias('Close'),
           df1['Volume'].cast('int').alias('Volume')
          ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### 8. Create a new dataframe with a column called HV Ration that is the ratio of the High Price versus volume of stock traded for a day.

In [58]:
df.select((df['High']/df['Volume']).alias('HV Rations')).show()

+--------------------+
|          HV Rations|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### 9. What day had the Peak High in Price?

In [59]:
df.orderBy(df['High'].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

#### 10. What is the mean of the Close column?

In [60]:
df.select(mean(df['Close']).alias('Mean of Close')).show()

+-----------------+
|    Mean of Close|
+-----------------+
|72.38844998012726|
+-----------------+



#### 11. What is the max and min of the Volume column?

In [61]:
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### 12. How many days was the Close lower than 60 dollars?

In [62]:
df.filter(df['Close'] < 60).count()

81

#### 13. What percentage of the time was the High greater than 80 dollars? In other words, (Number of Days High > 80) / (Total Days in the dataset)

In [63]:
df.filter(df['High'] > 80).count() / df.count() * 100

9.141494435612083

#### 14. What is the Pearson correlation between High and Volume?

In [64]:
df.select(corr('High','Volume').alias('Pearson correlation')).show()

+-------------------+
|Pearson correlation|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### 15. What is the max High per year?

In [65]:
df2 = df.withColumn('Year',year(df['Date']))
df3 = df2.groupBy('Year').max().select(['Year','max(High)'])
df3.show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### 16. What is the average Close for each Calendar Month? In other words, across all the years, what is the average Close price for Jan, Feb, Mar, etc... Your result will have a value for each of these months.

In [66]:
df4 = df.withColumn('Month', month('Date'))
df5 = df4.select('Month', 'Close').groupBy('Month').mean()
df5.select('Month','avg(Close)').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

